In [ ]:
import pandas as pd
import random
from datetime import datetime, timedelta

# Set random seed for reproducibility
random.seed(42)

# Manually defined list of course categories with fixed proportions
categories_full = [
    'Data Science', 'Data Science', 'Data Science',  # 3 courses = 30%
    'Business', 'Business', 'Business',              # 3 courses = 30%
    'Computer Science', 'Computer Science',          # 2 courses = 20%
    'Soft Skills', 'Language Learning'               # 2 courses = 20%
]

# Ensure we have exactly 10 courses
assert len(categories_full) == 10

# Possible course formats and their probabilities
formats = ['text', 'video', 'video+text']
format_weights = [0.25, 0.25, 0.5]

# Course payment type probabilities: 60% paid, 40% free
is_paid_weights = [0.6, 0.4]  # 1 = paid, 0 = free

# Generate synthetic data for courses
course_data = []

for i, category in enumerate(categories_full, start=1):
    title = f"Course {i}"  # Placeholder titles, to be renamed later
    format_choice = random.choices(formats, format_weights)[0]
    is_paid = random.choices([1, 0], weights=is_paid_weights)[0]
    duration_weeks = random.randint(2, 12)  # Course length in weeks
    author_id = random.randint(1, 25)       # Random author assignment

    course_data.append({
        "course_id": i,
        "title": title,
        "format": format_choice,
        "is_paid": is_paid,
        "duration_weeks": duration_weeks,
        "category": category,
        "author_id": author_id
    })

# Convert list of dictionaries to a DataFrame
courses_df = pd.DataFrame(course_data)

# Save the generated data to a CSV file
courses_df.to_csv("Courses_Table.csv", index=False)

In [ ]:
# Load course data from CSV
df = pd.read_csv("Courses_Table.csv")

# List of correct course titles — for the first 10 courses
titles = [
    "Data Analytics с нуля",
    "Введение в машинное обучение",
    "Погружение в статистику для аналитиков",
    "Основы стратегического менеджмента",
    "Маркетинг в цифровую эпоху",
    "Финансовая грамотность и личный бюджет",
    "Алгоритмы и структуры данных",
    "Основы Python-программирования",
    "Эффективное общение и публичные выступления",
    "Английский для начинающих: первые шаги"
]

# Replace default titles with proper ones for the first 10 courses
df.loc[:9, "title"] = titles

# Save the updated DataFrame to a new CSV file
df.to_csv("Courses_Table_updated.csv", index=False)

In [ ]:
# Parameters
num_users = 100
domains = ['gmail.com', 'mail.ru', 'yahoo.com']

# Realistic city distribution by frequency
cities_distribution = (
    ['Алматы'] * 25 + ['Астана'] * 20 + ['Шымкент'] * 15 + ['Караганда'] * 10 +
    ['Павлодар'] * 7 + ['Усть-Каменогорск'] * 6 + ['Актобе'] * 6 +
    ['Тараз'] * 5 + ['Жезказган', 'Атырау', 'Кокшетау'] * 2
)

# Age group distribution
age_groups = (
    [random.randint(18, 24) for _ in range(int(num_users * 0.35))] +  # 35%
    [random.randint(25, 34) for _ in range(int(num_users * 0.23))] +  # 23%
    [random.randint(35, 44) for _ in range(int(num_users * 0.20))] +  # 20%
    [random.randint(45, 60) for _ in range(num_users - int(num_users * 0.78))]  # 22%
)
random.shuffle(age_groups)

# Registration dates from May 1 to June 20, 2025 (51 days total)
reg_dates = [
    datetime(2025, 5, 1) + timedelta(days=random.randint(0, 50))
    for _ in range(num_users)
]

# Prepare user data list
users_data = []
used_emails = set()  # To ensure uniqueness of email addresses

# Name lists
kazakh_names = ['Aruzhan', 'Dias', 'Alua', 'Miras', 'Zarina', 'Nursultan', 'Aigerim', 'Yerzhan', 'Dana', 'Bauyrzhan',
                'Madi', 'Zhansaya', 'Nurzhan', 'Aruzat', 'Temirlan', 'Aidana', 'Alikhan', 'Diana', 'Sanzhar', 'Gulnaz']
russian_names = ['Andrei', 'Elena', 'Ivan', 'Maria', 'Dmitry', 'Olga', 'Sergey', 'Tatiana']

# Generate synthetic user data
for user_id in range(1, num_users + 1):
    is_kazakh = random.random() < 0.9  # 90% users are Kazakh
    name = random.choice(kazakh_names if is_kazakh else russian_names)
    gender = random.choices(['Male', 'Female'], weights=[0.55, 0.45])[0]
    age = age_groups[user_id - 1]

    # Generate birth date using age (safe with max 28 days)
    birth_year = 2025 - age
    birth_date = datetime(
        birth_year,
        random.randint(1, 12),
        random.randint(1, 28)
    ).date()

    city = random.choice(cities_distribution)
    reg_date = reg_dates[user_id - 1]

    # Generate unique email using only the name (no surname)
    email_base = name.lower()
    domain = random.choice(domains)
    email = f"{email_base}@{domain}"
    while email in used_emails:
        email = f"{email_base}{random.randint(1, 99)}@{domain}"
    used_emails.add(email)

    users_data.append({
        "user_id": user_id,
        "name": name,
        "gender": gender,
        "birth_date": birth_date,
        "email": email,
        "city": city,
        "registration_date": reg_date.date()
    })

# Convert to DataFrame and save as CSV
users_df = pd.DataFrame(users_data)
users_df.to_csv("Users_Table.csv", index=False)

In [ ]:
# Prepare data for Enrollments_Table
enrollments = []
enrollment_id = 1

for _, user in users_df.iterrows():
    user_id = user['user_id']
    reg_date = pd.to_datetime(user['registration_date'])

    # Each user enrolls in 1–3 courses, with weighted probability
    num_courses = random.choices([1, 2, 3], weights=[0.5, 0.3, 0.2])[0]
    enrolled_courses = random.sample(list(courses_df['course_id']), num_courses)

    for course_id in enrolled_courses:
        course = courses_df[courses_df['course_id'] == course_id].iloc[0]
        course_format = course['format']
        is_paid = course['is_paid']
        duration_weeks = course['duration_weeks']

        # Enrollment date: 0 to 10 days after registration
        enroll_date = reg_date + timedelta(days=random.randint(0, 10))
        if enroll_date > pd.Timestamp("2025-06-20"):
            enroll_date = pd.Timestamp("2025-06-20")

        # Base probability of completion depends on payment status
        if is_paid:
            base_completion_chance = random.uniform(0.60, 0.70)
        else:
            base_completion_chance = random.uniform(0.05, 0.15)

        # Adjust probability based on course format
        if course_format == 'video+text':
            base_completion_chance += 0.1
        elif course_format in ['video', 'text']:
            base_completion_chance -= 0.05

        # Adjust probability based on course length
        if duration_weeks <= 4:
            base_completion_chance += 0.1
        elif duration_weeks > 8:
            base_completion_chance -= 0.1

        # Clamp probability to [0, 1]
        base_completion_chance = min(1.0, max(0.0, base_completion_chance))

        # Determine completion based on probability
        completed = int(random.random() < base_completion_chance)

        # Generate progress: 100% if completed, else random 1–95%
        if completed:
            progress_percent = 100
        else:
            progress_percent = random.randint(1, 95)

        enrollments.append({
            "enrollment_id": enrollment_id,
            "user_id": user_id,
            "course_id": course_id,
            "enroll_date": enroll_date.date(),
            "completed": completed,
            "progress_percent": progress_percent
        })
        enrollment_id += 1

# Create DataFrame and save to CSV
enrollments_df = pd.DataFrame(enrollments)
enrollments_df.to_csv("Enrollments_Table.csv", index=False)

In [ ]:
# Device types with soft realistic distribution
device_types = ['Android', 'iOS', 'Windows', 'macOS', 'Linux']
device_weights = [0.39, 0.26, 0.19, 0.11, 0.05]  # intentionally non-strict percentages

# Function to generate a random datetime between two dates
def random_datetime_between(datetime_start, datetime_end):
    delta = datetime_end - datetime_start
    total_seconds = int(delta.total_seconds())
    random_seconds = random.randint(0, total_seconds)
    return datetime_start + timedelta(seconds=random_seconds)

# Session count distribution per user
session_distribution = (
    [random.randint(5, 8)] * int(num_users * 0.4) +     # 40% of users: active
    [random.randint(2, 4)] * int(num_users * 0.4) +     # 40%: moderate
    [1] * (num_users - int(num_users * 0.8))            # 20%: minimal
)
random.shuffle(session_distribution)

sessions = []
session_id = 1

# Assign sessions to users based on distribution
for user_row, num_sessions in zip(users_df.itertuples(index=False), session_distribution):
    user_id = user_row.user_id
    reg_date = pd.to_datetime(user_row.registration_date)

    for _ in range(num_sessions):
        # Randomly select device type by weighted probability
        device_type = random.choices(device_types, weights=device_weights)[0]

        # Generate login time between registration date and June 20, 2025
        login_date = random_datetime_between(
            datetime_start=reg_date,
            datetime_end=datetime(2025, 6, 20)
        )

        # Session duration depends on device type
        if device_type in ['Android', 'iOS']:
            duration = random.randint(5, 20)       # shorter sessions on mobile
        elif device_type in ['Windows', 'macOS']:
            duration = random.randint(15, 45)      # longer sessions on desktop
        else:  # Linux
            duration = random.randint(1, 30)       # varied behavior on Linux

        sessions.append({
            "session_id": session_id,
            "user_id": user_id,
            "device_type": device_type,
            "login_time": login_date,
            "session_duration_minutes": duration
        })
        session_id += 1

# Convert to DataFrame and export to CSV
sessions_df = pd.DataFrame(sessions)
sessions_df.to_csv("Sessions_Table.csv", index=False)


In [ ]:
# Merge to get 'is_paid' info for each enrollment
merged_df = enrollments_df.merge(courses_df[['course_id', 'is_paid']], on='course_id')

# Select only enrollments that were completed
completed_enrollments = merged_df[merged_df['completed'] == 1].copy()

# Sample comment texts for reviews
sample_comments = [
    "Очень полезный курс, спасибо!",
    "Ожидал большего, но в целом неплохо.",
    "Слишком поверхностно.",
    "Понравился формат и подача материала.",
    "Много практики, это хорошо.",
    "Сложный, но интересный материал.",
    "Немного скучный, но полезный.",
    "Советую всем новичкам.",
    "Курс помог с трудоустройством.",
    "Хорошее объяснение теории."
]

# Rating values and their selection probabilities
ratings = [5, 4, 3, 2, 1]
rating_weights = [0.4, 0.35, 0.15, 0.07, 0.03]

reviews = []
review_id = 1

for row in completed_enrollments.itertuples(index=False):
    is_paid = row.is_paid

    # Review probability: higher if course was paid
    leave_review_chance = 0.20 if is_paid else 0.15
    if random.random() <= leave_review_chance:
        # Assign rating based on weighted distribution
        rating = random.choices(ratings, weights=rating_weights)[0]

        # 13% chance to include a text comment
        if random.random() <= 0.13:
            comment = random.choice(sample_comments)
        else:
            comment = ""

        # Review date: within 0–7 days after enrollment
        enroll_date = pd.to_datetime(row.enroll_date)
        review_date = enroll_date + timedelta(days=random.randint(0, 7))
        if review_date > datetime(2025, 6, 20):
            review_date = datetime(2025, 6, 20)

        reviews.append({
            "review_id": review_id,
            "user_id": row.user_id,
            "course_id": row.course_id,
            "rating": rating,
            "comment": comment,
            "review_date": review_date.date()
        })
        review_id += 1

# Convert to DataFrame and export as CSV
reviews_df = pd.DataFrame(reviews)
reviews_df.to_csv("Reviews_Table.csv", index=False)

In [ ]:
# Read the Courses_Table which contains author_id
courses = pd.read_csv("Courses_Table.csv")

# Extract unique author IDs
author_ids = courses['author_id'].unique()

# Predefined nicknames and email domains
nicknames = [
    "DataNomad", "AlgoMaster", "CodeKazakh", "SteppeCoder",
    "QazaqByte", "NeuralNomad", "AstanaDev", "PavlodarGeek"
]

domains = ["@gmail.com", "@mail.ru", "@yandex.kz"]

# Build the Authors_Table DataFrame
authors = pd.DataFrame()
authors['author_id'] = author_ids
authors['nickname'] = nicknames[:len(author_ids)]  # Assign nicknames (as many as needed)
authors['email'] = [name.lower() + random.choice(domains) for name in authors['nickname']]

# Save the authors table to CSV
authors.to_csv("Authors_Table.csv", index=False)